In [1]:
import pandas as pd

try:
    users = pd.read_csv(r'C:\Users\BenBuczek\OneDrive - KIPP New Orleans, Inc\Desktop\Springboard\relax_challenge\takehome_users.csv', encoding='utf-8')
except UnicodeDecodeError:
    try:
        users = pd.read_csv(r'C:\Users\BenBuczek\OneDrive - KIPP New Orleans, Inc\Desktop\Springboard\relax_challenge\takehome_users.csv', encoding='latin1')
    except UnicodeDecodeError:
        users = pd.read_csv(r'C:\Users\BenBuczek\OneDrive - KIPP New Orleans, Inc\Desktop\Springboard\relax_challenge\takehome_users.csv', encoding='windows-1252')

print(users.head())

   object_id        creation_time               name  \
0          1  2014-04-22 03:53:30     Clausen August   
1          2  2013-11-15 03:45:04      Poole Matthew   
2          3  2013-03-19 23:14:52  Bottrill Mitchell   
3          4  2013-05-21 08:09:28    Clausen Nicklas   
4          5  2013-01-17 10:14:20          Raw Grace   

                        email creation_source  last_session_creation_time  \
0    AugustCClausen@yahoo.com    GUEST_INVITE                1.398139e+09   
1      MatthewPoole@gustr.com      ORG_INVITE                1.396238e+09   
2  MitchellBottrill@gustr.com      ORG_INVITE                1.363735e+09   
3   NicklasSClausen@yahoo.com    GUEST_INVITE                1.369210e+09   
4          GraceRaw@yahoo.com    GUEST_INVITE                1.358850e+09   

   opted_in_to_mailing_list  enabled_for_marketing_drip  org_id  \
0                         1                           0      11   
1                         0                           0       1   

In [2]:
user_engagement = pd.read_csv(r'C:\Users\BenBuczek\OneDrive - KIPP New Orleans, Inc\Desktop\Springboard\relax_challenge\takehome_user_engagement.csv')
print(user_engagement.head())

            time_stamp  user_id  visited
0  2014-04-22 03:53:30        1        1
1  2013-11-15 03:45:04        2        1
2  2013-11-29 03:45:04        2        1
3  2013-12-09 03:45:04        2        1
4  2013-12-25 03:45:04        2        1


In [3]:
import pandas as pd
from datetime import timedelta

user_engagement['time_stamp'] = pd.to_datetime(user_engagement['time_stamp'])

def get_adopted_users(df):
    df['week'] = df['time_stamp'].dt.to_period('W')
    weekly_logins = df.groupby(['user_id', 'week']).time_stamp.nunique()
    adopted_users = weekly_logins[weekly_logins >= 3].reset_index().user_id.unique()
    return adopted_users

adopted_users = get_adopted_users(user_engagement)

In [4]:
users['is_adopted'] = users['object_id'].isin(adopted_users).astype(int)

In [5]:
users['creation_time'] = pd.to_datetime(users['creation_time'])
users['last_session_creation_time'] = pd.to_datetime(users['last_session_creation_time'], unit='s')

users['account_age'] = (pd.to_datetime('today') - users['creation_time']).dt.days
users['days_since_last_session'] = (pd.to_datetime('today') - users['last_session_creation_time']).dt.days

users = pd.get_dummies(users, columns=['creation_source'])

C:\Users\BenBuczek\anaconda3\Lib\site-packages\pandas\core\tools\datetimes.py:557: RuntimeWarning: invalid value encountered in cast
  arr, tz_parsed = tslib.array_with_unit_to_datetime(arg, unit, errors=errors)


In [6]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

features = ['opted_in_to_mailing_list', 'enabled_for_marketing_drip', 'account_age', 'days_since_last_session'] + [col for col in users if col.startswith('creation_source_')]
X = users[features].fillna(0)
y = users['is_adopted']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LogisticRegression()
model.fit(X_train, y_train)

predictions = model.predict(X_test)
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.88      1.00      0.94      2108
           1       0.00      0.00      0.00       292

    accuracy                           0.88      2400
   macro avg       0.44      0.50      0.47      2400
weighted avg       0.77      0.88      0.82      2400



C:\Users\BenBuczek\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\BenBuczek\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\BenBuczek\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
